# Demo
### Taking input from the user, if it is a hot song, return another hot song, if not, return a similarly featured audio

In [516]:
from bs4 import BeautifulSoup
import requests
import json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

import pprint

import random 
from time import sleep

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle

In [517]:
Client_ID = config.client_id
Client_Secret = config.client_secret

sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = Client_ID, client_secret = Client_Secret))

In [579]:
#---------reading the scraped hot songs and songs from spotify from the csv file -----------#

df_hot = pd.read_csv('scrapped_hot_songs.csv')
clustered_df_original = pd.read_csv('the_clustered_spotify_songs.csv')

df_hot.head(10)

,song,artist
0,bad day,daniel powter
1,temperature,sean paul
2,promiscuous,nelly furtado featuring timbaland
3,you're beautiful,james blunt
4,hips don't lie,shakira featuring wyclef jean
5,unwritten,natasha bedingfield
6,crazy,gnarls barkley
7,ridin',chamillionaire featuring krayzie bone
8,sexyback,justin timberlake
9,check on it,beyonce featuring slim thug


In [580]:
#--------checking the similarity of input with the hot songs (checking for typos)------------#

from difflib import SequenceMatcher 
hot_song = df_hot['song'].values.tolist()


def similar(a):
    similarity = []
    for i in range (0,len(hot_song)):
        similarity.append(SequenceMatcher(a, hot_song).ratio())
    
    #most_similar = similarity.index(max(similarity))
    return similarity                                #hot_song[most_similar]           

In [519]:
#----------get the audio features of the suggested song by the---------#

def spotify_audio_features(user_song):   
    
    result = sp.search(user_song, limit=1) # Searching for the song in spotify
    result_id = result['tracks']['items'][0]['id']
    features = sp.audio_features(result_id)
    features_df = pd.DataFrame(features)
    return features_df

In [520]:
a = spotify_audio_features('hello')
a

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.905,0.647,10,-5.065,0,0.107,0.0187,0,0.282,0.367,130.97,audio_features,2r6OAV3WsYtXuXjvJ1lIDi,spotify:track:2r6OAV3WsYtXuXjvJ1lIDi,https://api.spotify.com/v1/tracks/2r6OAV3WsYtX...,https://api.spotify.com/v1/audio-analysis/2r6O...,190534,4


In [521]:
#----------dropping columns of the user song audio features ----------#

def trimmed_df (audio_features_df):
    audio_features_df = audio_features_df.drop(['time_signature','type', 'id', 'uri', 'track_href', 'analysis_url', 'mode', 'loudness', 'energy', 'instrumentalness', 'liveness' ], axis = 1)

    return audio_features_df
    

In [522]:
b = trimming_df(a)
b

,danceability,key,speechiness,acousticness,valence,tempo,duration_ms
0,0.905,10,0.107,0.0187,0.367,130.97,190534


In [523]:
with open('scaler_pickle', 'rb') as f:
    scaler = pickle.load(f) 
   

In [524]:
scaler.transform(b)

array([[ 1.81085827,  1.45041023,  0.49588705, -1.04799311, -0.12368614,
         0.56676125, -0.10957053]])

In [482]:
#----------cluster prediction of the user_song by calling the trained kmeans model ----------#


def cluster_prediction(audio_features_df):
    
    with open ('scaler_pickle', 'rb') as f:
        scaler = pickle.load(f)  
   
    with open ('kmeans_pickle', 'rb') as f:
        kmeans_model = pickle.load(f)
    
    scaled_audio_features_df = scaler.transform(audio_features_df) 
    cluster_prediction = kmeans_model.predict(scaled_audio_features_df)
    cluster_prediction = pd.Series(cluster_prediction).values[0]
    
    return cluster_prediction

In [459]:
c = cluster_prediction(b)
c, type(c)

(2, numpy.int32)

In [549]:
#----------suggesting a song with similar cluster ----------#
def choosing_song_within_same_cluster(input_song_cluster):
   
    clustered_df = clustered_df_original[clustered_df_original['cluster'] == input_song_cluster]
    suggestion = random.choice(clustered_df['song_name'].tolist())
                               
    return suggestion     

In [572]:
#--------Taking input and suggesting songs based on that ------------#

def user_song():
    input_song = input('Your Favorite Song: ')                       
    
    similarity = []
    if input_song.lower() in df_hot.song.values:                        #--------checking whether input is the hot song------#
        print('  Another Hot Song For You: ', random.choice(df_hot['song'].tolist()),'\n')
            
    #elif similar(input_song.lower()) in df_hot.song.values:              
    #    print('  Do You Mean: ', similar(input_song.lower()), '\n')     #--------checking for typos and check the hot songs again-------#
    #    user_song()
    
                                                                         #--------checking for songs again -------#
    elif (input_song.lower() in df_hot.song.values) == False:   
        song_feature_df = spotify_audio_features(input_song)
        for_prediction_df = trimmed_df(song_feature_df)
        input_song_cluster = cluster_prediction(for_prediction_df)
        suggestion = choosing_song_within_same_cluster(input_song_cluster)
        print("  Here, An Awesome Suggestion:", suggestion)
    
    else:
        print('  Sorry, Pick another song\n')
        user_song()

## Demo

In [576]:
user_song()

Your Favorite Song: thunderstruck
Here, An Awesome Suggestion: Stranglehold
